# Baseline w2v на слогах

В данном ноутбуке представлена только практическая часть применения word2vec. Неплохое введение в технологию на русском языке можно найти в данном [туториале](https://github.com/Yorko/mlcourse_open/blob/master/jupyter_russian/tutorials/word2vec_demonzheg.ipynb). Там же есть полезные ссылки на английском языке. Основная идея данного бейзлайна посмотрена оттуда.

Приступим:

In [1]:
# Импортируем библиотеки и функции
import pandas as pd
import numpy as np

from sklearn.metrics import roc_auc_score

In [2]:
# Загрузим обучающие данные
train = pd.read_csv('../data/train.csv')
print(train.shape)
train.head()

(101408, 2)


,Word,Label
0,Аалтонен,1
1,Аар,0
2,Аарон,0
3,ААРОН,0
4,Аарона,0


In [3]:
# Загрузим тестовые данные
test = pd.read_csv('../data/test.csv')
print(train.shape)
test.head()

(101408, 2)


,Word
0,Аалто
1,ААР
2,Аара
3,Ааре
4,Аарон


В данном подходе было решено основываться на слогах, так как на семинаре возникали некоторые идеи их применения. На просторах гитхаба была найдена [библиотека](https://github.com/tumikosha/iPoet), которая умеет разбивать слова на слоги, например:

In [4]:
from IPoet2 import slogTokenizer3

slogTokenizer3('Доблестный')

['До', 'бле', 'стный']

Однако стоит заметить, что данная библиотека не всегда срабатывает хорошо:

In [5]:
slogTokenizer3('Мальчик')

['Ма', 'льчик']

Это же слово на [slogi.su](http://slogi.su/%D0%BC%D0%B0%D0%BB%D1%8C%D1%87%D0%B8%D0%BA).

Вернемся к нашим данным. Для обучения модели w2v нам потребуются все слова, поэтому объединим обучающую и тестовые выборки:

In [6]:
data = pd.concat([train[['Word']], test[['Word']]])

Приведем их к нижнему регистру и оставим только кириллические символы, а также посчитаем кол-во гласных букв:

In [7]:
import re

def prepare_df(df):
    vowels = set('аоиеёэыуюя')
    
    df.loc[:, 'word'] = df['Word'].apply(lambda x: re.sub("[^а-я]", "", x.lower()))
    df.loc[:, 'vowels'] = df['word'].apply(lambda word: sum(letter in vowels for letter in word))
    
    return df

data = prepare_df(data)

Поссмотрим на слово с максимальным кол-вом слогов:

In [8]:
data[data['vowels']==data['vowels'].max()]

,Word,word,vowels
132521,Райффайзен Интернациональ Банк-Холдинг,райффайзенинтернациональбанкхолдинг,12


Будем считать, что фамилии, как правило, состоят из не более, чем 8 слогов. Напишем функцию, которая будет возвращать нам ровно 8 первых слогов. Если слогов меньше, то вместо них оставим пустые строки:

In [9]:
def slog(word):
    return (slogTokenizer3(word) + ['']*11)[:8]

data['list_w'] = data['word'].apply(slog)

In [10]:
data.head()

,Word,word,vowels,list_w
0,Аалтонен,аалтонен,4,"[а, а, лто, нен, , , , ]"
1,Аар,аар,2,"[а, ар, , , , , , ]"
2,Аарон,аарон,3,"[а, а, рон, , , , , ]"
3,ААРОН,аарон,3,"[а, а, рон, , , , , ]"
4,Аарона,аарона,4,"[а, а, ро, на, , , , ]"


Таким образом, построим w2v модель: предложением в нашем случае будет выступать последовательность слогов. Размер окна будет примем равным 3, а пространство для вектора слога примем равным 100.

In [11]:
from gensim.models import word2vec

model = word2vec.Word2Vec(data['list_w'], size=100, window=3, workers=4)

Создадим словарь со словами и соответсвующими им векторами:

In [12]:
w2v = dict(zip(model.wv.index2word, model.wv.syn0))

Т.к. сейчас мы каждому слогу сопоставили вектор, то нужно решить что сопоставить целому слово ("предложению" из слогов). Один из возможных вариантов это просто усреднить все вектора слов в предложении и получить некоторый смысл всего слова (если слова нет в тексте, то берем нулевой вектор).

In [13]:
class mean_vectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = len(next(iter(w2v.values())))
    
    def fit(self, X):
        return self 

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec] 
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

Получим представления обучающей выборке в виде последовательности слогов:

In [14]:
train = prepare_df(train)
train['list_w'] = train['word'].apply(slog)
train.head()

,Word,Label,word,vowels,list_w
0,Аалтонен,1,аалтонен,4,"[а, а, лто, нен, , , , ]"
1,Аар,0,аар,2,"[а, ар, , , , , , ]"
2,Аарон,0,аарон,3,"[а, а, рон, , , , , ]"
3,ААРОН,0,аарон,3,"[а, а, рон, , , , , ]"
4,Аарона,0,аарона,4,"[а, а, ро, на, , , , ]"


Теперь на основе этого получим представление обучающей выборки ввиде векторов:

In [15]:
train_matrix = mean_vectorizer(w2v).fit(train['list_w']).transform(train['list_w'])
print(train_matrix.shape)

(101408, 100)


Посмотрим результаты на валидационной выборке:

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    train_matrix, train['Label'].values.flatten(), 
    test_size=0.2, stratify=train['Label'], random_state=777)

lr_model = LogisticRegression().fit(X_train, y_train)

print(roc_auc_score(y_val, lr_model.predict_proba(X_val)[:, 1]))

0.7659296460897204


Аналогичным образом представим тестовую выборку в виде векторов:

In [17]:
test = prepare_df(test)
test['list_w'] = test['word'].apply(slog)
test_matrix = mean_vectorizer(w2v).fit(test['list_w']).transform(test['list_w'])
print(test_matrix.shape)

(188920, 100)


Обучим модель на всех данных и сделаем сабмит:

In [18]:
model = LogisticRegression().fit(train_matrix, train['Label'].values.flatten())

test['Prediction'] = model.predict_proba(test_matrix)[:, 0]
test = test.reset_index().rename(columns={'index': 'Id'})
test[['Id', 'Prediction']].to_csv('../submissions/sample_sub.csv', index=False)
test.head()

,Id,Word,word,vowels,list_w,Prediction
0,0,Аалто,аалто,3,"[а, а, лто, , , , , ]",0.778822
1,1,ААР,аар,2,"[а, ар, , , , , , ]",0.807601
2,2,Аара,аара,3,"[а, а, ра, , , , , ]",0.818267
3,3,Ааре,ааре,3,"[а, а, ре, , , , , ]",0.851415
4,4,Аарон,аарон,3,"[а, а, рон, , , , , ]",0.849097


На публичном лб значение метрики ROC AUC равно 0.76442, что в целом совпадает с нашей локальной валидацией. Относительно неплохой результат для бейзлайна.

На мой взгляд, можно поптаться его улучшить следующими способами:
* увеличить размерность векторного пространства, подобрать размер окна;
* придумать другой способ агрегации слогов (возможно, учитывающий каким-либо образом последовательность слогов);
* подобрать параметры модели;
* и многое другое ;)

Спасибо, что дочитали до конца. Буду рад обсуждению.